In [3]:
%pip install tiktoken
%pip install einops
%pip install accelerate
%pip install jsonlines

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import datasets
from datasets import load_dataset
import pandas as pd
import numpy as np
import jsonlines
import itertools
import torch


from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [5]:
kcc_dataset = pd.read_csv('/Users/ronaldmatovu/Downloads/Kisan Call Centre/kcc_dataset.csv')

/var/folders/0d/975d1jtj1v5692v1l52lsc5w0000gn/T/ipykernel_21088/2828159761.py:1: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  kcc_dataset = pd.read_csv('/Users/ronaldmatovu/Downloads/Kisan Call Centre/kcc_dataset.csv')


In [4]:
# select the first 1000 rows from the dataset
kcc_dataset_1000 = kcc_dataset[:1000]

In [5]:
kcc_dataset_1000.to_csv('data/kcc_dataset_1000.csv', index=False)

In [6]:
kcc_dataset_1000 = pd.read_csv('data/kcc_dataset_1000.csv')

In [7]:
kcc_dataset_1000.head()

,BlockName,Category,Year,Month,Day,Crop,DistrictName,QueryType,Season,Sector,StateName,QueryText,KccAns
0,0,0,2006,1,17,1275,SAGAR,99,RABI,HORTICULTURE,MADHYA PRADESH,how to control flower drop in bottelgourd,spray planofix4mlpump
1,0,0,2006,1,17,964,SAGAR,Disease Management,RABI,ANIMAL HUSBANDRY,MADHYA PRADESH,how tyo control diseases in buffalo,NaN
2,0,0,2006,1,17,1279,SAGAR,76,RABI,HORTICULTURE,MADHYA PRADESH,how to control fruit borer in brinjal,should be spray profenophos 35mlpump
3,0,0,2006,1,17,1064,SAGAR,3,RABI,AGRICULTURE,MADHYA PRADESH,how to control of yellow moisac in moong,should be spray metasystox 35mlpump
4,0,0,2006,1,17,1279,DAMOH,76,RABI,HORTICULTURE,MADHYA PRADESH,how to control white fly in brinjal,should be spray metasystox 35mlpump


In [8]:
kcc_dataset_1000 = kcc_dataset_1000.dropna()
kcc_dataset_1000 = kcc_dataset_1000.reset_index(drop=True)

In [9]:
kcc_dataset_1000.head()

,BlockName,Category,Year,Month,Day,Crop,DistrictName,QueryType,Season,Sector,StateName,QueryText,KccAns
0,0,0,2006,1,17,1275,SAGAR,99,RABI,HORTICULTURE,MADHYA PRADESH,how to control flower drop in bottelgourd,spray planofix4mlpump
1,0,0,2006,1,17,1279,SAGAR,76,RABI,HORTICULTURE,MADHYA PRADESH,how to control fruit borer in brinjal,should be spray profenophos 35mlpump
2,0,0,2006,1,17,1064,SAGAR,3,RABI,AGRICULTURE,MADHYA PRADESH,how to control of yellow moisac in moong,should be spray metasystox 35mlpump
3,0,0,2006,1,17,1279,DAMOH,76,RABI,HORTICULTURE,MADHYA PRADESH,how to control white fly in brinjal,should be spray metasystox 35mlpump
4,0,0,2006,1,17,Wheat,DAMOH,3,RABI,AGRICULTURE,MADHYA PRADESH,how to control termite in wheat,use chlorpyrephos1lithactwith the help of irri...


In [10]:
len(kcc_dataset_1000)

790

In [11]:
# Select the QueryText and KccAns columns
kcc_dataset_1000_QA = kcc_dataset_1000[['QueryText', 'KccAns']]

In [12]:
kcc_dataset_1000_QA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790 entries, 0 to 789
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   QueryText  790 non-null    object
 1   KccAns     790 non-null    object
dtypes: object(2)
memory usage: 12.5+ KB


In [13]:
kcc_dataset_1000_QA

,QueryText,KccAns
0,how to control flower drop in bottelgourd,spray planofix4mlpump
1,how to control fruit borer in brinjal,should be spray profenophos 35mlpump
2,how to control of yellow moisac in moong,should be spray metasystox 35mlpump
3,how to control white fly in brinjal,should be spray metasystox 35mlpump
4,how to control termite in wheat,use chlorpyrephos1lithactwith the help of irri...
...,...,...
785,ASKED ABOUT THE CONTROL OF BPH,RECOMENDED TO SPRAY CARBORYL 3gm LIT
786,ASKED ABOUT THE CONTROL OF TOBACCO CATERPILLAR,RECOMENDED TO SPRAY CHLORIPYRIPOS 2MLLIT
787,ASKED ABOUT THE CONTROL OF WILT,RECOMMENDED TO DRAIN OUT WATER IMMEDIATLYRAISE...
788,ASKED ABOUT THE CONTROL OF SHEATH BLIGHT,RECOMMENDED TO SPRAY HEXACONAZOLE 2mllit OF W...


In [14]:
kcc_dataset_1000_QA.to_csv('data/kcc_dataset_1000_QA.csv', index=False)

In [15]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import torch

In [26]:
# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [28]:
# Tokenize the input texts
def tokenize_function(examples):
    return tokenizer(examples['QueryText'], padding="max_length", truncation=True)

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
# trains split the data into training and validation sets with names (train_data and val_data)
# Path to the dataset: data/kcc_dataset_1000_QA.csv

dataset = pd.read_csv('data/kcc_dataset_1000_QA.csv')

# Filter rows with non-null answers
dataset_filtered = dataset.dropna(subset=['KccAns'])

# Select relevant columns for QA task
qa_dataset = dataset_filtered[['QueryText', 'KccAns']]

# Split the dataset into training and validation sets
train_data, val_data = train_test_split(qa_dataset, test_size=0.2, random_state=42)

In [41]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the input texts
def tokenize_function(example):
    input_text = example['QueryText'] + " " + tokenizer.eos_token + " " + example['KccAns']
    inputs = tokenizer(input_text, truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = inputs.input_ids.copy()  # Set labels to be the same as input_ids
    return inputs

# Tokenize the training and validation data
train_encodings = train_data.apply(tokenize_function, axis=1).tolist()
val_encodings = val_data.apply(tokenize_function, axis=1).tolist()

# Prepare datasets
class QADataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val) for key, val in self.encodings[idx].items()}

    def __len__(self):
        return len(self.encodings)

train_dataset = QADataset(train_encodings)
val_dataset = QADataset(val_encodings)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results_gpt2',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
metrics_gpt2 = trainer.evaluate()
print(metrics_gpt2)

# Save the fine-tuned model and tokenizer
model_save_path = './models/gpt2_fine_tuned'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


  0%|          | 0/3160 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 3.6706137657165527, 'eval_runtime': 3.8583, 'eval_samples_per_second': 40.951, 'eval_steps_per_second': 20.475, 'epoch': 1.0}


Checkpoint destination directory ./results_gpt2/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


...

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 4.872794151306152, 'eval_runtime': 3.6345, 'eval_samples_per_second': 43.473, 'eval_steps_per_second': 21.736, 'epoch': 10.0}
{'train_runtime': 833.3253, 'train_samples_per_second': 7.584, 'train_steps_per_second': 3.792, 'train_loss': 0.19197217286387577, 'epoch': 10.0}


  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 4.834618091583252, 'eval_runtime': 3.576, 'eval_samples_per_second': 44.183, 'eval_steps_per_second': 22.092, 'epoch': 10.0}


('./models/gpt2_fine_tuned/tokenizer_config.json',
 './models/gpt2_fine_tuned/special_tokens_map.json',
 './models/gpt2_fine_tuned/vocab.json',
 './models/gpt2_fine_tuned/merges.txt',
 './models/gpt2_fine_tuned/added_tokens.json')

In [42]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model_save_path = './models/gpt2_fine_tuned'  # Path to the saved model
tokenizer = GPT2Tokenizer.from_pretrained(model_save_path)
model = GPT2LMHeadModel.from_pretrained(model_save_path)

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

def generate_answer(question, max_length=50):
    # Tokenize the input question
    inputs = tokenizer.encode(question, return_tensors='pt')
    
    # Generate the answer
    outputs = model.generate(inputs, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    
    # Decode the generated answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Example question
question = "How to control fruit borer in brinjal?"

# Generate the answer
answer = generate_answer(question)
print("Question:", question)
print("Answer:", answer)

Question: How to control fruit borer in brinjal?
Answer: How to control fruit borer in brinjal?


In [43]:
print(generate_answer("how to control weed in field"))

how to control weed in field and field is the best to control weed in field and field is the best to control weed in field and field is the best to control weed in field and field is the best to control weed in field and field is the


In [14]:
examples = kcc_dataset_1000_QA.to_dict()
text = "Question: " + examples["QueryText"][0] + " Answer: " + examples["KccAns"][0]
text

'Question: how to control flower drop in bottelgourd Answer: spray planofix4mlpump'

In [15]:
examples["KccAns"][2]

'should be spray metasystox 35mlpump'

In [16]:
prompt_template_qa = """### Question:
{question}

### Answer:
{answer}"""

In [17]:
question = examples["QueryText"][0]
answer = examples["KccAns"][0]

text_with_prompt_template = prompt_template_qa.format(question=question, answer=answer)
text_with_prompt_template

'### Question:\nhow to control flower drop in bottelgourd\n\n### Answer:\nspray planofix4mlpump'

In [18]:
prompt_template_q = """### Question:
{question}

### Answer:"""

In [19]:
# Get the total number of examples in the dataset.
num_examples = len(examples["QueryText"])
num_examples

790

In [20]:
finetuning_dataset_text_only = []
finetuning_dataset_question_answer = []

In [21]:
for i in range(num_examples):
    question = examples["QueryText"][i]
    answer = examples["KccAns"][i]

    text_with_prompt_template_qa = prompt_template_qa.format(question=question, answer=answer)
    finetuning_dataset_text_only.append({"text": text_with_prompt_template_qa})

    text_with_prompt_template_q = prompt_template_q.format(question=question)
    finetuning_dataset_question_answer.append({"question": text_with_prompt_template_q, "answer": answer})

In [22]:
finetuning_dataset_text_only[0]

{'text': '### Question:\nhow to control flower drop in bottelgourd\n\n### Answer:\nspray planofix4mlpump'}

In [23]:
finetuning_dataset_question_answer[0]

{'question': '### Question:\nhow to control flower drop in bottelgourd\n\n### Answer:',
 'answer': 'spray planofix4mlpump'}

In [24]:
with jsonlines.open(f"data/kcc_qa_processed.jsonl", "w") as writer:
    writer.write_all(finetuning_dataset_question_answer)

In [25]:
finetuning_dataset = load_dataset('json', data_files='data/kcc_qa_processed.jsonl', split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [26]:
m = 5
print("Questio-Answer-fine-tuned dataset:")
top_m = list(itertools.islice(finetuning_dataset, m))
for j in top_m:
  print(j)

Questio-Answer-fine-tuned dataset:
{'question': '### Question:\nhow to control flower drop in bottelgourd\n\n### Answer:', 'answer': 'spray planofix4mlpump'}
{'question': '### Question:\nhow to control fruit borer in brinjal\n\n### Answer:', 'answer': 'should be spray profenophos 35mlpump'}
{'question': '### Question:\nhow to control of yellow moisac in moong\n\n### Answer:', 'answer': 'should be spray metasystox 35mlpump'}
{'question': '### Question:\nhow to control white fly in brinjal\n\n### Answer:', 'answer': 'should be spray metasystox 35mlpump'}
{'question': '### Question:\nhow to control termite in wheat\n\n### Answer:', 'answer': 'use chlorpyrephos1lithactwith the help of irrigation'}


In [47]:
# import the juypter notebook commons/common_functions.ipynb
%run commons/common_functions.ipynb

In [61]:
model_name = "gpt2"

In [62]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [63]:
filename = "data/kcc_qa_processed.jsonl"
instruction_dataset_df = pd.read_json(filename, lines=True)
examples = instruction_dataset_df.to_dict()

if "question" in examples and "answer" in examples:
  text = examples["question"][0] + examples["answer"][0]
elif "instruction" in examples and "response" in examples:
  text = examples["instruction"][0] + examples["response"][0]
elif "input" in examples and "output" in examples:
  text = examples["input"][0] + examples["output"][0]
else:
  text = examples["text"][0]

prompt_template = """### Question:
{question}

### Answer:"""

num_examples = len(examples["question"])
finetuning_dataset = []
for i in range(num_examples):
  question = examples["question"][i]
  answer = examples["answer"][i]
  text_with_prompt_template = prompt_template.format(question=question)
  finetuning_dataset.append({"question": text_with_prompt_template, "answer": answer})

from pprint import pprint
print("One datapoint in the finetuning dataset:")
pprint(finetuning_dataset[0])

One datapoint in the finetuning dataset:
{'answer': 'spray planofix4mlpump',
 'question': '### Question:\n'
             '### Question:\n'
             'how to control flower drop in bottelgourd\n'
             '\n'
             '### Answer:\n'
             '\n'
             '### Answer:'}


In [64]:
def tokenize_function(examples):
    if "question" in examples and "answer" in examples:
      text = examples["question"][0] + examples["answer"][0]
    elif "input" in examples and "output" in examples:
      text = examples["input"][0] + examples["output"][0]
    else:
      text = examples["text"][0]

    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        padding=True,
    )

    max_length = min(
        tokenized_inputs["input_ids"].shape[1],
        2048
    )
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=max_length
    )

    return tokenized_inputs

In [65]:
finetuning_dataset_loaded = datasets.load_dataset("json", data_files=filename, split="train")

tokenized_dataset = finetuning_dataset_loaded.map(
    tokenize_function,
    batched=True,
    batch_size=1,
    drop_last_batch=True
)

print(tokenized_dataset)

Map:   0%|          | 0/790 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 790
})


In [67]:
tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])

In [68]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 711
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 79
    })
})


In [69]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": True,
        "path": split_dataset
    },
    "verbose": True
}

In [128]:
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]
print("Train dataset:", train_dataset[0])
print("Test dataset:", test_dataset[0])

Train dataset: {'question': '### Question:\nwhat are control of cow pea mosaic \n\n### Answer:', 'answer': 'spray trizophose  30 ml pump', 'input_ids': [21017, 18233, 25, 198, 10919, 389, 1630, 286, 9875, 613, 64, 47076, 220, 198, 198, 21017, 23998, 25, 34975, 323, 1333, 89, 2522, 577, 220, 1542, 25962, 8901], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [21017, 18233, 25, 198, 10919, 389, 1630, 286, 9875, 613, 64, 47076, 220, 198, 198, 21017, 23998, 25, 34975, 323, 1333, 89, 2522, 577, 220, 1542, 25962, 8901]}
Test dataset: {'question': '### Question:\nASKED ABOUT CONTROL FOR WEEDS\n\n### Answer:', 'answer': 'RECOMMENDED TO SPRAY PENDIMETHYLIN 15LTACRE', 'input_ids': [21017, 18233, 25, 198, 1921, 42, 1961, 33478, 49833, 7473, 12887, 1961, 50, 198, 198, 21017, 23998, 25, 2200, 9858, 44, 49361, 5390, 6226, 30631, 350, 10619, 3955, 20702, 56, 34509, 1315, 27734, 2246, 2200], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1,

In [72]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [73]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

In [74]:
import logging
logger = logging.getLogger(__name__)
global_config = None

In [75]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

In [76]:
base_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [78]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [79]:
test_text = test_dataset[0]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {test_dataset[0]['answer']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question input (test): ### Question:
ASKED ABOUT CONTROL FOR WEEDS

### Answer:
Correct answer from Lamini docs: RECOMMENDED TO SPRAY PENDIMETHYLIN 15LTACRE
Model's answer: 


The problem with the "control" of weeds is that they are not controlled by the environment. They are controlled by the environment.

The problem with the "control" of weeds is that they are not controlled by the environment. They are controlled by the environment.

The problem with the "control" of weeds is that they are not controlled by the environment. They are controlled by the


In [117]:
max_steps = 3

In [118]:
trained_model_name = f"gpt2_{max_steps}_steps"
output_dir = trained_model_name

In [119]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM

In [120]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=100,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

In [108]:
model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
Memory footprint 0.510342192 GB
Flops 4180.672512 GFLOPs


In [121]:
from transformers import Trainer
trainer = Trainer(
    model=base_model,
    # model_flops=model_flops,
    # total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [122]:
training_output = trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

{'loss': 3.6347, 'learning_rate': 1e-05, 'epoch': 0.01}
{'loss': 3.0602, 'learning_rate': 5e-06, 'epoch': 0.01}
{'loss': 3.1144, 'learning_rate': 0.0, 'epoch': 0.02}
{'train_runtime': 2.6305, 'train_samples_per_second': 4.562, 'train_steps_per_second': 1.14, 'train_loss': 3.26977809270223, 'epoch': 0.02}


In [123]:
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

Saved model to: gpt2_3_steps/final


In [124]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)


In [125]:
finetuned_slightly_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [126]:
test_question = test_dataset[0]['question']
print("Question input (test):", test_question)

print("Finetuned slightly model's answer: ")
print(inference(test_question, finetuned_slightly_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question input (test): ### Question:
ASKED ABOUT CONTROL FOR WEEDS

### Answer:
Finetuned slightly model's answer: 


### Answer:

### Answer:

### Answer:

### Answer:

### Answer:

### Answer:

### Answer:

### Answer:

### Answer:

### Answer:

### Answer:

### Answer:

### Answer:

### Answer:

### Answer:

### Answer:




In [116]:
test_answer = test_dataset[0]['answer']
print("Target answer output (test):", test_answer)

Target answer output (test): RECOMMENDED TO SPRAY PENDIMETHYLIN 15LTACRE


In [48]:
use_hf = False

In [29]:
model_name = "gpt2"

In [49]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": finetuning_dataset
    },
    "verbose": True
}

In [55]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [56]:
test_question = finetuning_dataset[0]["question"]
test_question

'### Question:\nhow to control flower drop in bottelgourd\n\n### Answer:'

In [57]:
test_question = finetuning_dataset[0]["question"]
generated_answer = model_inference(test_question, model, tokenizer)
print(test_question)
print(generated_answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Question:
how to control flower drop in bottelgourd

### Answer:


The flower drop is a very important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most important part of the flower. It is the most imp

In [27]:
torch.random.manual_seed(0)
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [28]:
from huggingface_hub import login
login()

In [40]:
tokenizer = AutoTokenizer.from_pretrained(model_id, torch_dtype=torch.float16)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [41]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="save_folder"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [55]:
from transformers import RobertaForCausalLM, AutoTokenizer, pipeline

In [56]:
model_name = "deepset/roberta-base-squad2"

In [57]:
model = RobertaForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
finetuning_dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 790
})

In [59]:
sample_qa = finetuning_dataset[0]
sample_qa

{'question': '### Question:\nhow to control flower drop in bottelgourd\n\n### Answer:',
 'answer': 'spray planofix4mlpump'}

In [60]:
print(model_inference(sample_qa["question"], model, tokenizer))

aunchancyania CSV numericalagne tattancy STEPancy CSV numericalensenensen STEP numericalensenensen STEP numericalensen STEP numericalensen STEPensenancy CSV numericalagne CSV numericalagneancyrity Jurassicensen CSV numericalagneancy CSVancyusionsusionsagne CSVancyusionsancyusionsancyusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusionsusions rinkremlinurations STEPapoancy CSVagne Daneagne STEPapo realitiesagne CSV CSVagneancyagne STEP CSVagne CSV CSV spaambersambersagne CSV CSV CSVagneagneagne rocking STATS CSVagneancyagne CSV CSVagne paddedagne STEP CSVagne padded padded CSV realitiesremlinagne rink STATS CSVagne RESTagneereo STATSOTT STATS realities STATS realities STATS realities STATS realities STATS realitiesrity realities realities realities realitiesagne STATS realities STATS realities STATS realities rink STATS realities rink STATS rea